In [1]:
from qutree import *

In [42]:
# ch3 cart to eta
def build_C():
    mh = 1836.15267343
    m = np.array([12., 1., 1., 1.]) * mh
    M = np.sum(m)
    alpha = np.sqrt(m / M)
    N = len(m)
    C = np.zeros((N, N))
    C[:, 0] = alpha
    C[0, :] = alpha
    C[1:, 1:] = np.outer(alpha[1:] / (alpha[0] + 1), alpha[1:]) - np.eye(N-1)
    return C

def cart_to_spherical(x):
    r = np.linalg.norm(x, axis=0)
    theta = np.arccos(x[2] / r)
    phi = np.arctan2(x[1], x[0])
    return np.array([r, theta, phi])

def x_to_q(x):
    eta = build_C() @ x
    print('eta:', eta, eta.shape)
    r1, _, _ = cart_to_spherical(eta[1, :])
    r2, _, a = cart_to_spherical(eta[2, :])
    r3, t, b = cart_to_spherical(eta[3, :])
    chi = 0.5 * (a + b)
    phi = 0.5 * (-a + b)
#    print('a, b, chi, phi:', a, b, chi, phi)
#    print('r1, ...:', [r1, r2, r3, t, chi, phi])
    rho, vth, vphi = cart_to_spherical([r1, r2, r3])
    return np.array([rho, vth, vphi, t, chi, phi])

# Methyl fragment
C1 = np.array([0.0, 0.0, 0.0])
H1 = np.array([1.09, 0.0, 0.0])  # Along the x-axis
H2 = np.array([-0.545, 0.945, 0.0])  # Forming an angle of 120 degrees with the x-axis
H3 = np.array([-0.545, -0.945, 0.0])  # Forming an angle of 120 degrees with the x-axis
xyz_coordinates = np.array([C1, H1, H2, H3])

# small example
C = build_C()
q = x_to_q(xyz_coordinates)
print('q:', q)

eta: [[-1.16036073e-18  1.06700772e-17  0.00000000e+00]
 [-1.09000000e+00 -1.92384864e-18  0.00000000e+00]
 [ 5.45000000e-01 -9.45000000e-01  0.00000000e+00]
 [ 5.45000000e-01  9.45000000e-01  0.00000000e+00]] (4, 3)
q: [1.88896797 0.95512342 0.78580815 1.57079633 0.         1.0476707 ]


In [52]:
import autograd.numpy as np
# inverse trafo
def spherical_to_cart(q):
#    r, theta, phi = q
    sint = np.sin(q[1])
    x = q[0] * np.array([sint * np.cos(q[2]), sint * np.sin(q[2]), np.cos(q[1])])
    return np.array(x)

def q1tox(q):
    a = -q[5] + q[4]
    b =  q[5] + q[4]
    e0 = np.zeros(3)
    e1 = spherical_to_cart([q[0], q[3], 0])
    e2 = spherical_to_cart([q[1], q[3], a])
    e3 = spherical_to_cart([q[2], q[3], b])
    eta = np.stack([e0, e1, e2, e3])
    Cinv = np.linalg.solve(build_C(), np.eye(4))
    return Cinv @ eta

def q_to_x(q):
    qint = spherical_to_cart(q[:3])
    q = np.concatenate([qint, q[3:]])
    return q1tox(q).reshape(-1)

xref = np.array([[ 5.62873580e-01,  4.87995902e-01,  5.17273721e-17],
                 [-1.01328376e+00,  6.65108697e-02, -5.96931247e-17],
                 [-4.68283759e-01, -8.78489130e-01, -5.97478743e-17],
                 [-4.68283759e-01, -8.78489130e-01, -5.97478743e-17]])

q  = x_to_q(xref)
print('q:', q)
x2 = q_to_x(q)
print('|x - xref| = ', np.linalg.norm(x2 - xref.reshape(-1)))
# if you test points make sure both com and rot are set correctly in the xyz coords

eta: [[-1.04216860e-10  5.16089335e-10 -1.58943016e-26]
 [ 1.09000000e+00  9.18935974e-11  6.67432505e-17]
 [ 5.45000000e-01  9.45000000e-01  6.67980001e-17]
 [ 5.45000000e-01  9.45000000e-01  6.67980001e-17]] (4, 3)
q: [1.88896797 0.95512342 0.78580815 1.57079633 1.0476707  0.        ]
|x - xref| =  5.344659276211735e-10


In [54]:
import autograd.numpy as np
from autograd import jacobian
from functools import partial

def T_ij(q, i, j, x):
    J = jacobian(x)
    Jq = J(q)
    G = Jq.T @ Jq
    g = np.sqrt(np.linalg.det(G))
    return g
    #return G[i, j] * g

# compression of element with tn
r = 2
N = 10
dof = 6
i = 0
j = 0

prim = [linspace(0, 5, N),
        linspace(0, np.pi, N),
        linspace(0, 2*np.pi, N),
        linspace(0, np.pi, N),
        linspace(0, 2*np.pi, N),
        linspace(0, np.pi, N)]

# fot an element
T = partial(T_ij, i = i, j = j, x = q_to_x)
O = Objective(T, prim)
#G = balanced_tree(dof, r, N)
G = tt_graph(dof, r, N)
G = tn_grid(G, O.linspace)
Gopt, df = ttnopt(G, O, nsweep = 3)
fig = grid_animation(df)
fig.show()